## Conservation-based Synthetic Lethal Pair Search 
```
Title:  Conservation-based Synthetic Lethal Pair Search  
Authors: Taek-Kyun Kim  
Created: 02-07-2022   
Purpose: Retrive Synthetic Lethal Partners of The Genes in the Given List  Using Yeast Screen and Human-yeast Homology Information 
Notes: Runs in MyBinder
```

## Introduction

### Rationale

### Use-cases:
* Prioritize human candidate synthetic lethal interactions based on prior evidence of interaction in yeast SL screens
* _de novo_ discovery of SL interactions

### Approach
This notebook re-implements the approach outlined in Srivas et al. (2016)

### Usage:
Add genes of interest to "inputGenes" variable, then run the next step.

### Workflow Overview

### Datasets
#### Yeast Synthetic Lethal Interactions
Constanzo et al. (2016)
#### Human to Yeast Ortholog Mapping
How the Human to Yeast Ortholog Mapping data has been downloaded and saved into Bigquery tables can be found in the accompanying notebook (Mapping human to yeast orthologs)

### References
* Costanzo M, VanderSluis B, Koch EN, Baryshnikova A, Pons C, Tan G, Wang W, Usaj M, Hanchard J, Lee SD, Pelechano V, Styles EB, Billmann M, van Leeuwen J, van Dyk N, Lin ZY, Kuzmin E, Nelson J, Piotrowski JS, Srikumar T, Bahr S, Chen Y, Deshpande R, Kurat CF, Li SC, Li Z, Usaj MM, Okada H, Pascoe N, San Luis BJ, Sharifpoor S, Shuteriqi E, Simpkins SW, Snider J, Suresh HG, Tan Y, Zhu H, Malod-Dognin N, Janjic V, Przulj N, Troyanskaya OG, Stagljar I, Xia T, Ohya Y, Gingras AC, Raught B, Boutros M, Steinmetz LM, Moore CL, Rosebrock AP, Caudy AA, Myers CL, Andrews B, Boone C. **A global genetic interaction network maps a wiring diagram of cellular function.** Science. 2016 Sep 23;353(6306). pii: aaf1420. PubMed PMID: 27708008; PubMed Central PMCID: PMC5661885.
* Srivas R, Shen JP, Yang CC, Sun SM, Li J, Gross AM, Jensen J, Licon K, Bojorquez-Gomez A, Klepper K, Huang J, Pekin D, Xu JL, Yeerna H, Sivaganesh V, Kollenstart L, van Attikum H, Aza-Blanc P, Sobol RW, Ideker T. **A Network of Conserved Synthetic Lethal Interactions for Exploration of Precision Cancer Therapy**. Mol Cell. 2016 Aug 4;63(3):514-25. doi:10.1016/j.molcel.2016.06.022.Epub 2016 Jul 21. PubMed PMID: 27453043; PubMed Central PMCID: PMC5209245. 

## Preamble
This section describes how to setup the analysis environment, including google cloud platform authentication and import of all the necessary python libraries.

### Setup Analysis Environment

In [ ]:
#This code block installs the dependencies, please run it only once, the first time you run this notebook
#Please don't run it if you are running the notebook in MyBinder

!pip3 install google-cloud-bigquery
!pip3 install matplotlib
!pip3 install plotly
!pip3 install scipy
!pip3 install statsmodels
!pip3 install ipywidgets

In [1]:
# import modules
from google.cloud import bigquery
import sys
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
import ipywidgets as widgets
import plotly
import plotly.express as px
import pyarrow


## Google Authentication
Running the BigQuery cells in this notebook requires a Google Cloud Project, instructions for creating a project can be found in the [Google Documentation](https://cloud.google.com/resource-manager/docs/creating-managing-projects#console). The instance needs to be authorized to bill the project for queries.
For more information on getting started in the cloud see ['Quick Start Guide to ISB-CGC'](https://isb-cancer-genomics-cloud.readthedocs.io/en/latest/sections/HowToGetStartedonISB-CGC.html) and alternative authentication methods can be found in the [Google Documentation](https://googleapis.dev/python/google-api-core/latest/auth.html).

In [ ]:
#Please don't run this code block if you are running the notebook in MyBinder
#Users need to run the following command in their local machine or throught the notebook.
#Make sure to install the google cloud in the local envirionment. For more detail of gcloud installation, please see support from https://cloud.google.com/sdk/docs/install

!gcloud auth application-default login

In [3]:
# Replace syntheticlethality with your project ID
project_id='syntheticlethality'
client = bigquery.Client(project_id) 

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
%load_ext google.cloud.bigquery

In [ ]:
# a list for input genes 
inputGenes = ["DDX3X","DICER1","DROSHA","TNFRSF14","TRAF7","TSC1",'POLG',
              "FBXO11","PRDM1","RFWD3","AMER1","LZTR1","ATP2B3"]
inputGenes = ["'"+x+"'" for x in inputGenes]
inputGenes = ','.join(inputGenes)
inputGenes

## Map Yeast Orthologs & Get SL insteractions

In order to infer genetic interactions, the colony growth of double mutant strains is compared to that of single mutant strains as a measure of organismal fitness due to gene essentiality
.If the cell viability of a double mutant colony is higher or lower than that of two single mutants than positive or negative genetic interactions are inferred using a quantitative fitness metric or generic interaction score
.Synthetic lethal interactions are defined as the genetic interactions with low negative scores (< -0.35) at the extreme end of the interaction score distribution. Leveraging this dataset, we mapped yeast to human genes using yeast-human orthology information to identify presumed conserved human synthetic lethal pairs. The configurable parameters are listed as follows.

Find the synthetic lethal partners of the genes in the given list.

**Parameters**

**cutoff_algorithmMatchNo** is the desired minimum matching threshold required for a yeast-human gene comparison to be considered an ortholog.

**cutoff_score** The desired cutoff of the quantitative fitness metric. The default setting is (< 0.35) corresponding to the left tail of the distribution(<-0.35).

**cutoff_p** the desired significance threshold, the default value is p < 0.05.


In [6]:
sql = '''
WITH
--- Retrieve YeastSymbols mapped to HumanSymbols for the input genes
INPUT_H2Y AS (
  SELECT YeastSymbol
    FROM `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`
    
   WHERE HumanSymbol IN (__INPUTGENES__) AND
         AlgorithmsMatch >= __ALGORITHMCUTOFF__
),

--- Identify protein-protein interactions using the YeastSymbols (left match)
Yeast_ITX1 AS (
  SELECT UPPER(Query_allele_name)       AS Interactor1, 
         UPPER(Array_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Query_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Identify protein-protein interactions using the YeastSymbols (right match)
Yeast_ITX2 AS (
  SELECT UPPER(Array_allele_name)       AS Interactor1, 
         UPPER(Query_allele_name)       AS Interactor2,
         Genetic_interaction_score_____ AS Interaction_score,
         P_value
    FROM `isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions`
   WHERE (Genetic_interaction_score_____ < __SCORECUTOFF__ AND P_value < __PvalueCUTOFF__) AND
         (UPPER(Array_allele_name) IN (SELECT YeastSymbol FROM INPUT_H2Y))
   
),

--- Union interaction tables
Union_ITX AS (
  SELECT * FROM Yeast_ITX1
   UNION ALL
  SELECT * FROM Yeast_ITX2
)

--- Convert YeastSymbols to HumanSymbols in the protein-protein interations
SELECT DISTINCT 
       GINFO1.EntrezID        AS EntrezID_Input,
       H2Y1.HumanSymbol       AS Gene_Input,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor1       AS Gene_Input_Yeast,
       GINFO2.EntrezID        AS EntrezID_SL_Candidate,
       H2Y2.HumanSymbol       AS Gene_SL_Candidate,
---       Add if you want to know what yeast genes are involved
---       YITX.Interactor2       AS Gene_SL_Candidate_Yeast,
       YITX.Interaction_score AS Interaction_score,
       YITX.P_value           AS P_value
       
  FROM Union_ITX AS YITX
       LEFT JOIN `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`                      AS H2Y1   ON YITX.Interactor1 = H2Y1.YeastSymbol
       LEFT JOIN `isb-cgc-bq.annotations_versioned.Human2Yeast_mapping_Alliance_for_Genome_Resources_R3_0_1`                      AS H2Y2   ON YITX.Interactor2 = H2Y2.YeastSymbol
       LEFT JOIN `isb-cgc-bq.synthetic_lethality.gene_info_human_HGNC_NCBI_2020_07` AS GINFO1 ON H2Y1.HumanID = GINFO1.HGNCID
       LEFT JOIN  `isb-cgc-bq.synthetic_lethality.gene_info_human_HGNC_NCBI_2020_07` AS GINFO2 ON H2Y2.HumanID = GINFO2.HGNCID
       
 WHERE (H2Y1.HumanSymbol IS NOT NULL AND YITX.Interactor1 IS NOT NULL) AND
       (H2Y2.HumanSymbol IS NOT NULL AND YITX.Interactor2 IS NOT NULL)

'''
# select the thresholds to be used
cutoff_algorithmMatchNo = "3"
cutoff_score = "-0.35"
cutoff_p = "0.01"

sql = sql.replace("__INPUTGENES__", inputGenes)
sql = sql.replace("__ALGORITHMCUTOFF__", cutoff_algorithmMatchNo)
sql = sql.replace("__SCORECUTOFF__", cutoff_score)
sql = sql.replace("__PvalueCUTOFF__", cutoff_p)

res = client.query(sql).to_dataframe()



## Get Yeast SL Interactions

In [7]:
# List the SL partner genes for the input genes
res


,EntrezID_Input,Gene_Input,EntrezID_SL_Candidate,Gene_SL_Candidate,Interaction_score,P_value
0,54514,DDX4,4117,MAK,-0.432,0.005777
1,54514,DDX4,22858,CILK1,-0.432,0.005777
2,8653,DDX3Y,4117,MAK,-0.432,0.005777
3,8653,DDX3Y,22858,CILK1,-0.432,0.005777
4,1654,DDX3X,4117,MAK,-0.432,0.005777
5,1654,DDX3X,22858,CILK1,-0.432,0.005777


**Gene_Input**  The user's input gene list whose synthetic lethal partner(s) are seeked. 

**EntrezID_Input** shows the Entrez ids of the genes in the user's input gene list 

**EntrezID_SL_Candidate and Gene_SL_Candidate** are the Entrez ids and gene symbols for the inferred synthetic lethal partners. 

**Interaction_score and P_value**  the estimate of interaction strength between input gene and its SL partner in the isb-cgc-bq.supplementary_tables.Constanzo_etal_Science_2016_SGA_Genetic_Interactions table. 



The results can be saved to a csv file.

In [ ]:
res.to_csv(path_or_buf='conserved_SL_output.csv', index=False)
